In [1]:
import os.path

import h5py
import pandas as pd
import yaml

In [2]:
# load data from image files according the the yaml file
path = ["..","..","run_data","2023_11_17"]

run_data = yaml.safe_load(open(os.path.join(*path, "data.yml")))

In [3]:
run_data = pd.DataFrame(run_data)
run_data

,ICT1,ICT2,dipole_name,quad_pv_name,save_filename,tdc_name
0,"[0.9685895525, 0.9275407734, 0.9227464728, 1.0...","[1.0794398194, 1.0766765703, 1.064339215, 1.18...",0,0.0,../run_data/2023_11_17/13ARV1_1701286176.h5,0
1,"[1.0386552651, 0.9595960648, 0.9009403263, 0.9...","[0.8815740854, 1.1187479521, 0.8820179623, 0.9...",0,2.5,../run_data/2023_11_17/13ARV1_1701286193.h5,0
2,"[0.9575771563, 1.0550407668, 0.9911021657, 0.9...","[0.9897631782, 0.9026718179, 0.9613517121, 0.9...",0,5.0,../run_data/2023_11_17/13ARV1_1701286211.h5,0
3,"[0.9754499864, 1.0685999139, 1.0424194168, 0.9...","[1.0082677283, 1.0056026229, 0.9788066444, 1.1...",0,7.5,../run_data/2023_11_17/13ARV1_1701286225.h5,0
4,"[0.9641839214, 1.0028211861, 0.9202209618, 1.0...","[1.0924266447, 1.1035695579, 0.9492391804, 0.9...",0,10.0,../run_data/2023_11_17/13ARV1_1701286243.h5,0
5,"[0.9845524484, 0.9159889088, 0.9463392509, 0.9...","[1.0172486335, 1.0470616554, 0.9528940276, 0.9...",0,0.0,../run_data/2023_11_17/13ARV1_1701286256.h5,1
6,"[0.9260447723, 0.9442116863, 1.0852642445, 0.9...","[0.8998327749, 0.9407936466, 0.9652159014, 1.2...",0,2.5,../run_data/2023_11_17/13ARV1_1701286272.h5,1
7,"[1.0097759096, 1.0452899895, 1.041574279, 1.01...","[1.0678310221, 0.9919742227, 0.9988086481, 1.0...",0,5.0,../run_data/2023_11_17/13ARV1_1701286289.h5,1
8,"[0.9642815874, 1.0737504077, 0.9136486462, 1.0...","[0.8777402427, 0.9402199595, 0.9244413054, 0.9...",0,7.5,../run_data/2023_11_17/13ARV1_1701286309.h5,1
9,"[1.0041341996, 0.997799832, 0.9456453785, 1.06...","[1.0395839434, 1.0899779246, 1.0193278183, 0.9...",0,10.0,../run_data/2023_11_17/13ARV1_1701286324.h5,1


In [4]:
# create a pivot table out of the data to collect settings
pivot_data = pd.pivot(
    run_data, index=["tdc_name", "dipole_name"],
    columns="quad_pv_name",
    values="save_filename"
)

In [5]:
# get numpy values in the correct shape
num_rows, num_cols = pivot_data.shape

fnames = pivot_data.values.reshape(2,2,5)

In [6]:
# create function to get data from h5 files
import numpy as np
def read_file(fname):
    with h5py.File(fname) as f:
        data_x = np.array((
            f["images"].attrs["quad_pv_name"],
            f["images"].attrs["tdc_name"],
            f["images"].attrs["dipole_name"],
        ))
        images = f["images"][:]

    return data_x, images

In [7]:
train_x = []
train_y = []
for i in range(len(fnames)):
    tx = []
    ty = []
    for j in range(len(fnames[0])):
        ttx = []
        tty = []
        for k in range(len(fnames[0,0])):
            data_x, images = read_file("../"+fnames[i,j,k])

            print(data_x)
            print(images.shape)

            ttx += [data_x]
            tty += [images]

        tx += [ttx]
        ty += [tty]

    train_x += [tx]
    train_y += [ty]

train_x = np.array(train_x).transpose([2,0,1,3])
train_y = np.array(train_y).transpose([2,0,1,3,4,5])

##
# params: [number of quad strengths, number of tdc voltages (2, off/on), number of dipole angles (2, off/on),
# number of scanning elements (3)]
# images: [number of quad strengths, number of tdc voltages (2, off/on), number of
# dipole angles (2, off/on), number of images per parameter configuration, width in pixels, height in pixels]

[0. 0. 0.]
(10, 2000, 2000)
[2.5 0.  0. ]
(10, 2000, 2000)
[5. 0. 0.]
(10, 2000, 2000)
[7.5 0.  0. ]
(10, 2000, 2000)
[10.  0.  0.]
(10, 2000, 2000)
[0. 0. 2.]
(10, 2000, 2000)
[2.5 0.  2. ]
(10, 2000, 2000)
[5. 0. 2.]
(10, 2000, 2000)
[7.5 0.  2. ]
(10, 2000, 2000)
[10.  0.  2.]
(10, 2000, 2000)
[0. 1. 0.]
(10, 2000, 2000)
[2.5 1.  0. ]
(10, 2000, 2000)
[5. 1. 0.]
(10, 2000, 2000)
[7.5 1.  0. ]
(10, 2000, 2000)
[10.  1.  0.]
(10, 2000, 2000)
[0. 1. 2.]
(10, 2000, 2000)
[2.5 1.  2. ]
(10, 2000, 2000)
[5. 1. 2.]
(10, 2000, 2000)
[7.5 1.  2. ]
(10, 2000, 2000)
[10.  1.  2.]
(10, 2000, 2000)


In [8]:
train_x.shape

(5, 2, 2, 3)

In [9]:
train_y.shape


(5, 2, 2, 10, 2000, 2000)

In [10]:
train_x[0]

array([[[0., 0., 0.],
        [0., 0., 2.]],

       [[0., 1., 0.],
        [0., 1., 2.]]])